# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtest.utils import run_strategy, run_wfo


from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random

random.seed(42)

In [3]:
initial_cash = 10_000
margin = 1/30
commission = 7e-4

In [16]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()

groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [18]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if ('Forex_group' in symbol.path or 'Forex_Indicator' in symbol.path) and symbol.spread < 10]
# tickers = ['USOILm']

print(tickers)

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['USDBND', 'USDJOD', 'USDKWD', 'USDSAR', 'USDHRK', 'USDBNDm', 'USDKWDm', 'USDHRKm', 'AUDCHFm', 'AUDUSDm', 'CADCHFm', 'EURUSDm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
USDBND
USDBND
USDJOD
USDJOD
USDKWD
USDKWD
USDSAR
USDSAR
USDHRK
se descarto USDHRK
USDHRK
se descarto USDHRK
USDBNDm
USDBNDm
USDKWDm
USDKWDm
USDHRKm
se descarto USDHRKm
USDHRKm
se descarto USDHRKm
AUDCHFm
AUDCHFm
AUDUSDm
AUDUSDm
CADCHFm
CADCHFm
EURUSDm
EURUSDm


In [19]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

{16388: Timestamp('2021-10-28 08:00:00+0000', tz='UTC'),
 16386: Timestamp('2021-10-28 10:00:00+0000', tz='UTC')}

In [20]:
start_date_train = Timestamp('2023-09-01 00:00:00', tz='UTC')
limit_date_train = Timestamp('2024-09-01 00:00:00', tz='UTC')

print(start_date_train)
print(limit_date_train)

2023-09-01 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Analisis preliminar

In [21]:
from backtesting import Strategy
import talib as ta
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class MeanReversion(Strategy):
    pip_value = None
    minimum_lot = None
    contract_volume = None
    minimum_units = None
    
    sma_period = 50
    deviation_threshold = 0.01
    risk = 1
    cum_rsi_up_threshold = 90
    cum_rsi_down_threshold = 10
    atr_multiplier = 2
    
    def init(self):
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.sma_period)
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        
        self.minimum_units = self.minimum_lot * self.contract_volume


    def next(self):
        # Precio actual y valor de la SMA
        actual_close = self.data.Close[-1]
        sma_value = self.sma[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]

        # Desviación del precio con respecto a la SMA (en porcentaje)
        deviation = (actual_close - sma_value) / sma_value

        if self.position:
            if self.position.is_long and actual_close >= self.sma:
                self.position.close()

            if self.position.is_short and actual_close <= self.sma:
                self.position.close()  
        
        else:
            # Condiciones para comprar (precio por debajo de la SMA más del umbral de desviación)
            if deviation <= -self.deviation_threshold and cum_rsi <= self.cum_rsi_down_threshold:
                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                if units < self.minimum_units:
                    units = self.minimum_units
                
                self.buy(
                    size=units,
                    sl=sl_price
                )

            # Condiciones para vender (precio por encima de la SMA más del umbral de desviación)
            elif deviation >= self.deviation_threshold and cum_rsi >= self.cum_rsi_up_threshold:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                if units < self.minimum_units:
                    units = self.minimum_units
                
                self.sell(
                    size=units,
                    sl=sl_price
                )

In [22]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    MeanReversion
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)

        df_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=commission, 
            prices=symbols[ticker][interval], 
            initial_cash=initial_cash, 
            margin=1/30
        )

        performance = pd.concat([performance, df_stats])

    except:
        print(f'hubo un problema con {ticker} {interval}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AUDCHFm 16388
AUDCHFm 16386
AUDUSDm 16388
AUDUSDm 16386
CADCHFm 16388
CADCHFm 16386
EURUSDm 16388
EURUSDm 16386
USDBND 16388
USDBND 16386
USDBNDm 16388
USDBNDm 16386
USDJOD 16388
USDJOD 16386
USDKWD 16388
USDKWD 16386
USDKWDm 16388
USDKWDm 16386
USDSAR 16388
USDSAR 16386


In [24]:

filter_performance = performance[
    (performance['return/dd']>1)
    # & (performance['stability_ratio']>0.7)
].sort_values(by=['return/dd'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric


In [ ]:
for ticker, interval in zip(portfolio, intervals):
    df_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=commission, 
        prices=symbols[ticker][interval], 
        initial_cash=initial_cash, 
        margin=1/30,
        plot=True
    )


# WFO

In [ ]:
from backtest.utils import optim_func_2

strategy = MeanReversion
lookback_bars = 1000
validation_bars = 100
warmup_bars = 50

all_wfo_performances = pd.DataFrame()
stats_per_symbol = {}

params = {
    'deviation_threshold': list(np.arange(0.01, 0.2, 0.01).round(2)),
    'sma_period': [50],
}

for index, row in filter_performance.iterrows():
    
    # try:
    ticker = row.ticker
    interval = row.interval
    
    print(ticker, interval)
    
    prices = symbols[ticker][interval]

    wfo_stats, df_stats = run_wfo(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        prices=prices,
        initial_cash=initial_cash,
        commission=commission,
        margin=margin,
        optim_func=optim_func_2,
        params=params,
        lookback_bars=lookback_bars,
        warmup_bars=warmup_bars,
        validation_bars=validation_bars
    )
    
    if ticker not in stats_per_symbol.keys():
        stats_per_symbol[ticker] = {}

    stats_per_symbol[ticker][interval] = wfo_stats

    all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    # except:
    #     print(f'No se pudo ejecutar para el ticker {ticker}')

all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

all_wfo_performances.sort_values(by='return/dd', ascending=False)

# Montecarlo

In [ ]:
filtered_wfo_performance = all_wfo_performances[
    (all_wfo_performances['stability_ratio'] > 0.7)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

In [18]:
# stats_per_symbol['UKOILm'][16386]['_trades']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from backtest.utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=initial_cash, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
